In [1]:
%cd ..

/home/eli/AnacondaProjects/categorical_bpl


In [2]:
import argparse
import collections
import pyro
import torch
import numpy as np
import data_loader.data_loaders as module_data
import model.model as module_arch
from parse_config import ConfigParser
from trainer import Trainer

RDKit WARNING: [11:09:05] Enabling RDKit 2019.09.3 jupyter extensions


In [3]:
%matplotlib inline

In [4]:
# pyro.enable_validation(True)
# torch.autograd.set_detect_anomaly(True)

In [5]:
# fix random seeds for reproducibility
SEED = 123
torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
np.random.seed(SEED)

In [6]:
Args = collections.namedtuple('Args', 'config resume device')
config = ConfigParser.from_args(Args(config='chemical_config.json', resume=None, device=None))

In [7]:
logger = config.get_logger('train')

In [8]:
# setup data_loader instances
data_loader = config.init_obj('data_loader', module_data)
valid_data_loader = data_loader.split_validation()

In [9]:
# build model architecture, then print to console
model = config.init_obj('arch', module_arch)

In [10]:
optimizer = pyro.optim.ReduceLROnPlateau({
    'optimizer': torch.optim.Adam,
    'optim_args': {
        "lr": 1e-4,
        "weight_decay": 0,
        "amsgrad": True
    },
    "patience": 50,
    "factor": 0.1,
    "verbose": True,
})

In [11]:
# optimizer = config.init_obj('optimizer', pyro.optim)

In [12]:
trainer = Trainer(model, [], optimizer, config=config,
                  data_loader=data_loader,
                  valid_data_loader=valid_data_loader,
                  lr_scheduler=optimizer, log_images=False)

In [13]:
trainer.train()

Train Epoch: 1 [128/225000 (0%)] Loss: 55128.839844
Train Epoch: 1 [1536/225000 (1%)] Loss: 54572.738281
Train Epoch: 1 [2944/225000 (1%)] Loss: 51793.761719
Train Epoch: 1 [4352/225000 (2%)] Loss: 49305.718750
Train Epoch: 1 [5760/225000 (3%)] Loss: 49255.472656
Train Epoch: 1 [7168/225000 (3%)] Loss: 47818.671875
Train Epoch: 1 [8576/225000 (4%)] Loss: 37596.101562
Train Epoch: 1 [9984/225000 (4%)] Loss: 36286.707031
Train Epoch: 1 [11392/225000 (5%)] Loss: 36477.785156
Train Epoch: 1 [12800/225000 (6%)] Loss: 40157.867188
Train Epoch: 1 [14208/225000 (6%)] Loss: 34400.484375
Train Epoch: 1 [15616/225000 (7%)] Loss: 44552.707031
Train Epoch: 1 [17024/225000 (8%)] Loss: 25144.384766
Train Epoch: 1 [18432/225000 (8%)] Loss: 37204.828125
Train Epoch: 1 [19840/225000 (9%)] Loss: 23136.542969
Train Epoch: 1 [21248/225000 (9%)] Loss: 20360.332031
Train Epoch: 1 [22656/225000 (10%)] Loss: 24250.691406
Train Epoch: 1 [24064/225000 (11%)] Loss: 19880.162109
Train Epoch: 1 [25472/225000 (11%)]

In [14]:
model.cpu()

MolecularVaeCategoryModel(
  (_category): FreeCategory(
    (generator_0): MolecularDecoder(
      (pre_recurrence_linear): Sequential(
        (0): Linear(in_features=196, out_features=196, bias=True)
        (1): SELU()
      )
      (recurrence): GRU(196, 50, num_layers=3, batch_first=True)
      (decoder): Sequential(
        (0): Linear(in_features=50, out_features=34, bias=True)
        (1): LogSoftmax(dim=-1)
      )
    )
    (generator_0_dagger): ConvMolecularEncoder(
      (smiles_conv): Sequential(
        (0): Conv1d(120, 9, kernel_size=(9,), stride=(1,))
        (1): ReLU()
        (2): Conv1d(9, 9, kernel_size=(9,), stride=(1,))
        (3): ReLU()
        (4): Conv1d(9, 10, kernel_size=(11,), stride=(1,))
        (5): ReLU()
      )
      (smiles_linear): Sequential(
        (0): Linear(in_features=80, out_features=196, bias=True)
        (1): SELU()
      )
      (embedding_loc): Linear(in_features=196, out_features=196, bias=True)
      (embedding_log_scale): Linear(in

In [15]:
valid_xs, valid_ys = list(valid_data_loader)[-1]

In [16]:
m, recons = model(observations=valid_xs, train=False)

In [17]:
(recons == valid_xs).all(dim=-1).flatten().to(dtype=torch.float).mean()

tensor(0.6519)